# Mencoba Implementasi Regresi Linear
Sebelumnya kita telah mempelajari ide dari training, gradient descent, dan backpropagation. Pada bagian ini kita akan mencoba mengimplementasikan regresi linear, termasuk membahas tentang data pipeline, model, loss function, dan optimizer menggunakan stochastic gradient descent.

### Import Library yang Digunakan

In [ ]:
import random
import torch
import matplotlib.pyplot as plt

### Membuat fungsi untuk generate data sintesis
Data sintetis yang dibuat dari fungsi ini adalah sebagai berikut:

In [ ]:
def datasintetis(w, b, jumlah_data):
    X = torch.normal(0, 1, (jumlah_data, len(w))) # bangkitkan data acak sejumlah `jumlah_data`
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.1, y.shape)
    return X, y.reshape((-1, 1))

### Membangkitkan data sintesis dengan mendefinisikan target *weight* dan *bias*

In [ ]:
# w dan b asli hanya digunakan untuk generate data sintesis
w_asli = torch.tensor([2, -3.4])
b_asli = 4.2

# generate data sintesis sejumlah 1000 data berdasarkan w dan b asli
# dalam hal ini $y = 2x1 - 3.4x2 + 4.2$
X, y = datasintetis(w_asli, b_asli, 1000)
print(X.shape, y.shape)

# scatter plot data sintesis
plt.scatter(X[:, (1)].detach().numpy(), y.detach().numpy(), 1)

### Membaca Dataset

In [ ]:
def memuat_data(batch_size, X, y):
    # menghitung panjang data masukan
    banyaknya_data = len(X)
    # memberikan indeks dari data yang akan dijadikan mini-batch
    indices = list(range(banyaknya_data))
    # mengacak urutan data
    random.shuffle(indices)

    for i in range(0, banyaknya_data, batch_size):
        batch_indices = torch.tensor(indices[i: min(i + batch_size, banyaknya_data)])
        yield X[batch_indices], y[batch_indices]


batch_size = 10

for X, y in memuat_data(batch_size, X, y):
    print(X, '\n', y)

Yield adalah salah satu cara kita untuk mengembalikan sebuah value dan menghentikan sementara eksekusi function yang sedang berjalan. Biasanya ekspresi `yield` digunakan dalam [fungsi yang bersifat sebagai generator](https://docs.python.org/3/glossary.html#term-generator)

Lihat lebih lanjut mengenai ekspresi `yield` pada Python [disni](https://docs.python.org/3/reference/expressions.html#yieldexpr)

In [ ]:
def simpleGeneratorFun():
    yield 1
    yield 2
    yield 3

# fungsi generator dipanggil berkali-kali untuk membangkitkan data hingga eksperi yield yang paling akhir pada fungsi generator 
for value in simpleGeneratorFun():
    print("Hasil dari yield adalah: ", value)


def simpleGenerator():
    yield 9
    return 8


for value in simpleGenerator():
    print("Hasil dari return adalah: ", value) # hanya mengembalikan nilai pada ekspresi yield, nilai yang di return tidak akan ditampilkan

### Inisiialisasi Weight, Bias, Model, Loss Function dan Optimizer

In [ ]:
# inisiasi weight dan bias
w = torch.normal(0, 0.01, size=(2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)


# menentukan model linear regression
def linreg(X, w, b):
    return torch.matmul(X, w) + b # model regresi linear $y = Xw + b$, X adalah data dan y adalah output regresi


# menentukan loss function
def squared_loss(y_hat, y):
    return (y_hat - y.view(y_hat.shape)) ** 2 / 2 # mean squared error loss


# menentukan optimizer
def sgd(params, lr, batch_size):
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad / batch_size # update weight dan bias menggunakan stochastic gradient descent
            param.grad.zero_()

### Melakukan Training

In [ ]:
# menentukan learning rate dan epoch
lr = 0.01
epoch = 3

for epoch in range(epoch):
    for X, y in memuat_data(batch_size, X, y):
        # menghitung prediksi
        nilai_loss = squared_loss(linreg(X, w, b), y)
        # menghitung gradien
        nilai_loss.sum().backward()
        # mengupdate nilai weight dan bias
        sgd([w, b], lr, batch_size)
    with torch.no_grad():
        # menampilkan nilai loss
        print('Epoch {}, nilai loss: {:.2f}, nilai w: {:.2f}, nilai b: {:.2f}'
              .format(epoch + 1, nilai_loss.sum().item(), w[0].item(), b.item()))

Karena kita mengetahui nilai weight dan bias yang sesungguhnya, kita dapat mengkalkulasi prediksi yang benar dengan menggunakan model yang sudah kita buat.

In [ ]:
# Menghitung error terhadap nilai weight dan bias yang sesungguhnya
print("Error dalam menghitung w: ", (w_asli - w.reshape(w_asli.shape)).data.numpy())
print("Error dalam menghitung b: ", (b - b_asli).norm().item())